In [45]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb

df = pd.read_csv('big_data_fix.csv')
print(len(df))
df.head()

13073


,0,1,2,3,4,5
0,Совещательный орган при императоре в начале XX в,Сенат,Государственный совет,Консилия министров,Верховный тайный совет,2
1,Министр внутренних дел с 1904 г либерал автор ...,Лорис Меликов,Святополк Мирский,Милюков,Витте,2
2,Какое из названных событий произошло 27 января...,подрыв флагманского корабля Петропавловск,высадка японских войск на Ляодунском полуострове,героический бой Варяга и Корейца,заключение Портсмутского мирного договора,3
3,Что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,Творчество поэтов Гумилева Ахматовой Мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1


In [40]:
import pickle

with open('/Users/user/Python/Diploma/Embeddings/elmo_embed_big.pickle', 'rb') as f:
    elmo_embs = pickle.load(f)

elmo_embs = np.array(elmo_embs)

with open('/Users/user/Python/Diploma/Embeddings/fasttext_embed_big.pickle', 'rb') as f:
    fasttext_embs = pickle.load(f)

fasttext_embs = np.array(fasttext_embs)

with open('/Users/user/Python/Diploma/Embeddings/rusvec_embed_big.pickle', 'rb') as f:
    rusvec_embs = pickle.load(f)

rusvec_embs = np.array(rusvec_embs)

with open('/Users/user/Python/Diploma/Embeddings/bert_embed_big.pickle', 'rb') as f:
    bert_embs = pickle.load(f)

bert_embs = np.array(bert_embs)

In [41]:
embeddings = {'RusVectores': rusvec_embs, 'FastText': fasttext_embs, 'ELMO': elmo_embs, 'BERT': bert_embs}

In [42]:
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import accuracy_score, precision_score, f1_score

def count_cosine_similarity(data, embeddings):
    cosines = np.zeros((embeddings.shape[0], embeddings.shape[1]-1))
    for i in range(embeddings.shape[0]):
        for j in range(1,5):
            cosines[i, j-1] = cos(embeddings[i,0].reshape(1,-1), embeddings[i,j].reshape(1,-1))[0]
            
    preds = np.argmax(cosines, axis=1)
    preds = preds + 1
    target = np.array(data['5'], int)
    
    accuracy = accuracy_score(target, preds)
    precision_macro = precision_score(target, preds, average='macro')
    precision_micro = precision_score(target, preds, average='micro')
    f1_macro = f1_score(target, preds, average='macro')
    f1_micro = f1_score(target, preds, average='micro')
    
    return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

In [43]:
results_dict = {}
for key, embed in embeddings.items():
    results_dict[key] = count_cosine_similarity(df, embed)

In [44]:
results = pd.DataFrame(data=results_dict, columns=list(embeddings.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,RusVectores,FastText,ELMO,BERT
Accuracy,0.301461,0.318672,0.308881,0.273464
Precision Macro,0.302205,0.318558,0.308875,0.273466
Precision Micro,0.301461,0.318672,0.308881,0.273464
F1 Macro,0.301348,0.318392,0.308720,0.273504
F1 Micro,0.301461,0.318672,0.308881,0.273464
